<a href="https://colab.research.google.com/github/ashivashankars/CMPE256_Assignments/blob/main/Electronic_Retailer_Market_Basket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Electronic Retailer Market Basket
Recommendation System for Online
Checkout Integration


# Task
Design and implement a Market Basket Recommendation System for an electronic retailer using historical transaction data. The system should dynamically recommend products based on the items currently in the customer's cart, handle cases where no patterns exist for the current item combination, and utilize Gradio for the user interface. The final output should include the trained model and a Gradio application.

## Data loading and exploration

### Subtask:
Load the historical transaction data and explore its structure and content to understand the data we are working with.


**Reasoning**:
The first step is to load the data and explore its structure and content as per the instructions. I will import pandas, load the data from the specified path, display the head, info, and descriptive statistics of the dataframe in a single code block.



In [1]:
import pandas as pd

# Assuming the data file is located at '/data/transactions.csv' based on typical data science environments
# If the actual file path is different, this will need to be adjusted.
try:
    df = pd.read_csv('/content/CMPE256_Hackathon_market_basket_analysis_Release.csv')
except FileNotFoundError:
    # Handle the case where the file is not found in the expected location
    # For this example, I'll assume a different path for demonstration purposes if the first one fails
    # In a real scenario, you might need to ask the user for the correct path or handle it differently
    print("File not found at /data/transactions.csv. Attempting to load from a different path.")
    try:
        df = pd.read_csv('/kaggle/input/electronics-store-transactions/transactions.csv')
    except FileNotFoundError:
         print("File not found at /kaggle/input/electronics-store-transactions/transactions.csv either. Please ensure the file is in the correct directory.")
         df = None # Set df to None to indicate loading failed


if df is not None:
    print("DataFrame head:")
    display(df.head())

    print("\nDataFrame info:")
    display(df.info())

    print("\nDataFrame description:")
    display(df.describe())

DataFrame head:


,transaction_id,item_1,item_2,item_3,item_4,item_5
0,1,Honeywell 5800CO Carbon Monoxide Detector (SKU...,Hanwha XRN-2010 NVR (SKU: XRN-2010),NaN,NaN,NaN
1,2,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Bosch B5512 Control Panel (SKU: B5512),Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES),GE Interlogix 60-807-95R Glassbreak Detector (...,Bosch F220-B6 Detector Base (SKU: F220-B6)
2,3,Bosch F220-B6 Detector Base (SKU: F220-B6),GE Interlogix 60-652-95R Carbon Monoxide Detec...,DSC WS4933 Carbon Monoxide Detector (SKU: WS4933),NaN,NaN
3,4,GE Interlogix 60-652-95R Carbon Monoxide Detec...,Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2),Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...,Bosch B5512 Control Panel (SKU: B5512)
4,5,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US),DSC WS4916 Smoke Detector (SKU: WS4916),Axis P3367-VE Network Camera (SKU: 0407-001),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  1000 non-null   int64 
 1   item_1          1000 non-null   object
 2   item_2          1000 non-null   object
 3   item_3          763 non-null    object
 4   item_4          535 non-null    object
 5   item_5          277 non-null    object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


None


DataFrame description:


,transaction_id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


In [2]:
try:
    df = pd.read_csv('/content/CMPE256_Hackathon_market_basket_analysis_Release.csv')
    print("DataFrame head:")
    display(df.head())

    print("\nDataFrame info:")
    display(df.info())

    print("\nDataFrame description:")
    display(df.describe())

except FileNotFoundError:
    print("File not found at /content/sample_data/california_housing_train.csv. Please ensure the file is in the correct directory.")
    df = None # Set df to None to indicate loading failed


DataFrame head:


,transaction_id,item_1,item_2,item_3,item_4,item_5
0,1,Honeywell 5800CO Carbon Monoxide Detector (SKU...,Hanwha XRN-2010 NVR (SKU: XRN-2010),NaN,NaN,NaN
1,2,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Bosch B5512 Control Panel (SKU: B5512),Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES),GE Interlogix 60-807-95R Glassbreak Detector (...,Bosch F220-B6 Detector Base (SKU: F220-B6)
2,3,Bosch F220-B6 Detector Base (SKU: F220-B6),GE Interlogix 60-652-95R Carbon Monoxide Detec...,DSC WS4933 Carbon Monoxide Detector (SKU: WS4933),NaN,NaN
3,4,GE Interlogix 60-652-95R Carbon Monoxide Detec...,Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2),Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...,Bosch B5512 Control Panel (SKU: B5512)
4,5,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US),DSC WS4916 Smoke Detector (SKU: WS4916),Axis P3367-VE Network Camera (SKU: 0407-001),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   transaction_id  1000 non-null   int64 
 1   item_1          1000 non-null   object
 2   item_2          1000 non-null   object
 3   item_3          763 non-null    object
 4   item_4          535 non-null    object
 5   item_5          277 non-null    object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


None


DataFrame description:


,transaction_id
count,1000.000000
mean,500.500000
std,288.819436
min,1.000000
25%,250.750000
50%,500.500000
75%,750.250000
max,1000.000000


## Data preprocessing

### Subtask:
Prepare the data for market basket analysis by transforming it into a suitable format, such as a one-hot encoded DataFrame or a list of transactions.


**Reasoning**:
Inspect the DataFrame structure and transform the data into a list of transactions, cleaning the item names.



In [3]:
# 1. Inspect the structure of the df DataFrame (already done in the previous step, but we can quickly re-verify)
print("DataFrame head:")
display(df.head())

# 2. Transform the data into a list of lists
transactions = []
item_columns = [col for col in df.columns if col.startswith('item_')]
for index, row in df.iterrows():
    transaction = [str(item).strip() for item in row[item_columns].dropna().tolist()]
    transactions.append(transaction)

# 3. Remove any leading or trailing whitespace from the item names (done in step 2 during list comprehension)

# 4. Display the first few transactions
print("\nFirst 5 transactions:")
for i in range(min(5, len(transactions))):
    print(transactions[i])

DataFrame head:


,transaction_id,item_1,item_2,item_3,item_4,item_5
0,1,Honeywell 5800CO Carbon Monoxide Detector (SKU...,Hanwha XRN-2010 NVR (SKU: XRN-2010),NaN,NaN,NaN
1,2,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Bosch B5512 Control Panel (SKU: B5512),Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES),GE Interlogix 60-807-95R Glassbreak Detector (...,Bosch F220-B6 Detector Base (SKU: F220-B6)
2,3,Bosch F220-B6 Detector Base (SKU: F220-B6),GE Interlogix 60-652-95R Carbon Monoxide Detec...,DSC WS4933 Carbon Monoxide Detector (SKU: WS4933),NaN,NaN
3,4,GE Interlogix 60-652-95R Carbon Monoxide Detec...,Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2),Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...,Bosch B5512 Control Panel (SKU: B5512)
4,5,Hanwha QNV-6010R Network Camera (SKU: QNV-6010R),Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US),DSC WS4916 Smoke Detector (SKU: WS4916),Axis P3367-VE Network Camera (SKU: 0407-001),Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-I...



First 5 transactions:
['Honeywell 5800CO Carbon Monoxide Detector (SKU: 5800CO)', 'Hanwha XRN-2010 NVR (SKU: XRN-2010)']
['Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Bosch B5512 Control Panel (SKU: B5512)', 'Pelco IMP1110-1ES IP Camera (SKU: IMP1110-1ES)', 'GE Interlogix 60-807-95R Glassbreak Detector (SKU: 60-807-95R)', 'Bosch F220-B6 Detector Base (SKU: F220-B6)']
['Bosch F220-B6 Detector Base (SKU: F220-B6)', 'GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)', 'DSC WS4933 Carbon Monoxide Detector (SKU: WS4933)']
['GE Interlogix 60-652-95R Carbon Monoxide Detector (SKU: 60-652-95R)', 'Dahua 8-Channel NVR (SKU: NVR4208-8P-4KS2)', 'Bosch NDN-50022-A3 IP Camera (SKU: NDN-50022-A3)', 'Dahua DH-IPC-HDBW4431R-ZS IP Camera (SKU: DH-IPC-HDBW4431R-ZS)', 'Bosch B5512 Control Panel (SKU: B5512)']
['Hanwha QNV-6010R Network Camera (SKU: QNV-6010R)', 'Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US)', 'DSC WS4916 Smoke Detector (SKU: WS4916)', 'Axis P3367-VE Network Camer

## Market basket analysis

### Subtask:
Apply an algorithm like Apriori or FP-growth to find frequent itemsets and association rules from the preprocessed data.


In [5]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 2. Convert the list of transactions (transactions) into a one-hot encoded DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# 3. Apply the Apriori algorithm to find frequent itemsets
# Experiment with min_support values (e.g., 0.01, 0.05)
frequent_itemsets = apriori(df_onehot, min_support=0.01, use_colnames=True)

# 4. Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="confidence")

# Filter the rules by a minimum confidence threshold (e.g., 0.5 or 0.7)
rules = rules[rules['confidence'] >= 0.5]


# 5. Display the first few rows of the generated frequent itemsets and association rules DataFrames
print("Frequent Itemsets:")
display(frequent_itemsets.head())

print("\nAssociation Rules:")
display(rules.head())

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Frequent Itemsets:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
0,0.080,(Axis M3046-V Network Camera (SKU: 0806-001))
1,0.089,(Axis P3225-LVE Network Camera (SKU: 0935-001))
2,0.090,(Axis P3367-VE Network Camera (SKU: 0407-001))
3,0.092,(Bosch B5512 Control Panel (SKU: B5512))
4,0.087,(Bosch B810 Wireless Receiver (SKU: B810))



Association Rules:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

**Reasoning**:
The previous command failed because `association_rules` does not have a `min_confidence` argument directly. The confidence threshold is applied to the resulting rules DataFrame. I will generate the rules first and then filter by confidence.



In [7]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# 2. Convert the list of transactions (transactions) into a one-hot encoded DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# 3. Apply the Apriori algorithm to find frequent itemsets
# Experiment with min_support values (e.g., 0.01, 0.05)
frequent_itemsets = apriori(df_onehot, min_support=0.005, use_colnames=True)

# 4. Generate association rules from the frequent itemsets
rules = association_rules(frequent_itemsets, metric="confidence")

# Filter the rules by a minimum confidence threshold (e.g., 0.5 or 0.7)
rules = rules[rules['confidence'] >= 0.1]


# 5. Display the first few rows of the generated frequent itemsets and association rules DataFrames
print("Frequent Itemsets:")
display(frequent_itemsets.head())

print("\nAssociation Rules (filtered by confidence >= 0.1):")
display(rules.head())

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Frequent Itemsets:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
0,0.080,(Axis M3046-V Network Camera (SKU: 0806-001))
1,0.089,(Axis P3225-LVE Network Camera (SKU: 0935-001))
2,0.090,(Axis P3367-VE Network Camera (SKU: 0407-001))
3,0.092,(Bosch B5512 Control Panel (SKU: B5512))
4,0.087,(Bosch B810 Wireless Receiver (SKU: B810))



Association Rules (filtered by confidence >= 0.1):


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


The previous step successfully generated frequent itemsets, but no association rules were found with a minimum confidence of 0.5. This could be due to the dataset or the chosen threshold. I will try a lower minimum confidence value to see if any rules are generated.



In [8]:
# 4. Generate association rules from the frequent itemsets with a lower minimum confidence
rules = association_rules(frequent_itemsets, metric="confidence")

# Filter the rules by a lower minimum confidence threshold (e.g., 0.1 or 0.2)
rules = rules[rules['confidence'] >= 0.1]


# 5. Display the first few rows of the generated frequent itemsets and association rules DataFrames
print("Frequent Itemsets:")
display(frequent_itemsets.head())

print("\nAssociation Rules (filtered by confidence >= 0.1):")
display(rules.head())

Frequent Itemsets:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
0,0.080,(Axis M3046-V Network Camera (SKU: 0806-001))
1,0.089,(Axis P3225-LVE Network Camera (SKU: 0935-001))
2,0.090,(Axis P3367-VE Network Camera (SKU: 0407-001))
3,0.092,(Bosch B5512 Control Panel (SKU: B5512))
4,0.087,(Bosch B810 Wireless Receiver (SKU: B810))



Association Rules (filtered by confidence >= 0.1):


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


Lowering the confidence threshold to 0.1 still resulted in no association rules. This suggests that there might not be strong associations between items in this dataset at the current minimum support level. I will try lowering the minimum support threshold for the Apriori algorithm to see if more frequent itemsets are found, which might then lead to some association rules.



In [9]:
# 3. Apply the Apriori algorithm to find frequent itemsets with a lower minimum support
# Experiment with min_support values (e.g., 0.005 or lower)
frequent_itemsets = apriori(df_onehot, min_support=0.005, use_colnames=True)

# 4. Generate association rules from the frequent itemsets with a minimum confidence of 0.1
rules = association_rules(frequent_itemsets, metric="confidence")

# Filter the rules by a minimum confidence threshold (e.g., 0.1)
rules = rules[rules['confidence'] >= 0.1]


# 5. Display the first few rows of the generated frequent itemsets and association rules DataFrames
print("Frequent Itemsets:")
display(frequent_itemsets.head())

print("\nAssociation Rules (filtered by confidence >= 0.1):")
display(rules.head())

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

Frequent Itemsets:


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

,support,itemsets
0,0.080,(Axis M3046-V Network Camera (SKU: 0806-001))
1,0.089,(Axis P3225-LVE Network Camera (SKU: 0935-001))
2,0.090,(Axis P3367-VE Network Camera (SKU: 0407-001))
3,0.092,(Bosch B5512 Control Panel (SKU: B5512))
4,0.087,(Bosch B810 Wireless Receiver (SKU: B810))


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


Association Rules (filtered by confidence >= 0.1):


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Recommendation system implementation

### Subtask:
Build a recommendation function that takes a list of items in the cart as input and uses the association rules to recommend additional items.



Define the `recommend_items` function to recommend items based on the association rules and the items in the cart.



In [10]:
def recommend_items(cart_items, rules):
    """
    Recommends items based on association rules and items in the customer's cart.

    Args:
        cart_items: A list of strings representing items in the customer's cart.
        rules: A DataFrame containing the association rules.

    Returns:
        A list of unique recommended items.
    """
    if rules.empty:
        print("No recommendations can be generated due to a lack of association rules.")
        return []

    recommended_items = set()
    for index, rule in rules.iterrows():
        # Convert frozenset antecedents to a set for easier checking
        antecedents = set(rule['antecedents'])
        # Check if all antecedents are in the cart
        if antecedents.issubset(set(cart_items)):
            # Add consequents to recommended items if not already in the cart
            for consequent_item in rule['consequents']:
                if consequent_item not in cart_items:
                    recommended_items.add(consequent_item)

    return list(recommended_items)

# Example usage (assuming 'rules' DataFrame is available from previous steps)
# sample_cart = ['Axis P3367-VE Network Camera (SKU: 0407-001)', 'Pelco DSSRV2-040-US NVR (SKU: DSSRV2-040-US)']
# recommendations = recommend_items(sample_cart, rules)
# print("\nRecommended items for the sample cart:")
# print(recommendations)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Gradio interface development

### Subtask:
Create a Gradio interface that allows users to interact with the recommendation system by adding items to a virtual cart and receiving real-time recommendations.


**Reasoning**:
Import the gradio module and create a list of all unique items from the original dataframe.



In [12]:
import gradio as gr

# Create a list of all unique items
unique_items = set()
item_columns = [col for col in df.columns if col.startswith('item_')]
for _, row in df.iterrows():
    for item in row[item_columns].dropna().tolist():
        unique_items.add(item.strip())

unique_items_list = sorted(list(unique_items))
#print(f"Number of unique items: {len(unique_items_list)}")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

**Reasoning**:
Define the get_recommendations function and create and launch the Gradio interface using the unique items list and the defined function.



In [13]:
!pip install --upgrade jupyter_client

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 4.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

  Attempting uninstall: jupyter_client
    Found existing installation: jupyter_client 7.4.9
    Uninstalling jupyter_client-7.4.9:
      Successfully uninstalled jupyter_client-7.4.9


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.3 which is incompatible.
jupyter-kernel-gateway 2.5.2 requires jupyter-client<8.0,>=5.2.0, but you have jupyter-client 8.6.3 which is incompatible.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
def get_recommendations(cart_items, rules_df):
    """
    Recommends items based on association rules and items in the customer's cart,
    formatted for Gradio output.

    Args:
        cart_items: A list of strings representing items selected in the Gradio interface.
        rules_df: A DataFrame containing the association rules.

    Returns:
        A formatted string of unique recommended items or a message if no
        recommendations are found.
    """
    if rules_df.empty:
        return "No recommendations can be generated due to a lack of association rules."

    recommended_items = set()
    for index, rule in rules_df.iterrows():
        # Convert frozenset antecedents to a set for easier checking
        antecedents = set(rule['antecedents'])
        # Check if all antecedents are in the cart
        if antecedents.issubset(set(cart_items)):
            # Add consequents to recommended items if not already in the cart
            for consequent_item in rule['consequents']:
                if consequent_item not in cart_items:
                    recommended_items.add(consequent_item)

    if recommended_items:
        return "Recommended Items:\n" + "\n".join(list(recommended_items))
    else:
        return "No recommendations for this combination of items."

# Create the Gradio Interface
iface = gr.Interface(
    fn=lambda items: get_recommendations(items, rules), # Pass the rules DataFrame to the function
    inputs=gr.CheckboxGroup(unique_items_list, label="Items in Cart"),
    outputs=gr.Textbox(label="Recommended Items"),
    title="Electronic Retailer Market Basket Recommender",
    description="Select items in your cart to get product recommendations."
)

# Launch the Gradio interface
iface.launch(debug=True)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

* Running on public URL: https://2ba8e9acfac7aa03da.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag